# INPUTS

In [19]:
start_point=[30.624801,75.549551]  #Bottom-left point of box
end_point=[31.167619,76.180482] #Top-right point of box
api_key=''  ##INSERT YOUR GOOGLE API KEY HERE

dic_coor=[30.893299,75.864912] 


In [84]:
import requests
import matplotlib.pyplot as plt
from skimage import io
import os
import cv2
import numpy as np
%matplotlib inline

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import json
import pandas as pd

import datetime
from IPython.display import clear_output

In [34]:
def progressing(start_time,cur_step,total_steps):
    bar_length=50
    cur_time=datetime.datetime.today()
    
    cur_step +=1 ##because loop starts from 0
    elapsed=cur_time-start_time
    total=elapsed/cur_step *total_steps
    left=total-elapsed
    clear_output(wait=True)

    print('Total elapsed: ',elapsed.__str__())
    if cur_step == total_steps:
        print('Time left: 0:00:00.0000')
    else:
        print('Time left: ',left.__str__())
    print('Total Time: ',total.__str__())
    print('\nProgress: [','#'*(int(cur_step/total_steps*bar_length)),'-'*(bar_length-int(cur_step/total_steps*bar_length)),']',np.round(cur_step/total_steps*100,decimals=4),'%')

    
    


In [60]:

width=500
height=300

##DON'T CHANGE THESE VALUES
#lat_dist=0.005406999999998163
#long_dist=0.01268699999999967-0.001973
lat_dist=0.005495000000003358
long_dist=0.01075500000001739
zoom=16


crop_section=0.0004505833333331803
size=str(width)+'x'+str(height)

In [85]:
##area of 1 px in km^2 
px_area=((30.659516000000004 - 30.654021 )*111)/height*((76.27289400000002 - 76.262139)*97)/width 
px_area

px_lat=((30.659516000000004 - 30.654021 )*111)/height
px_long=((76.27289400000002 - 76.262139)*97)/width 



In [22]:
start_lat,end_lat=sorted([start_point[0],end_point[0]])

start_long,end_long=sorted([start_point[1],end_point[1]])

In [23]:
def number(a,b,dist):
    tmp=a
    counter=0
    while tmp< b:
        tmp += dist
        counter +=1
    return counter

In [24]:
h_num=number(start_long,end_long,long_dist)
v_num=number(start_lat,end_lat,lat_dist)

In [25]:
h_num,v_num

(59, 99)

In [26]:
print('Total number of images generated:',h_num*v_num)

Total number of images generated: 5841


# DATA GENERATION

In [80]:
#First run this cell
latitude_list=[start_lat,start_lat+lat_dist]
longitude_list=[start_long,start_long+long_dist]
counter=1
zz=0

directory = os.path.abspath('images')
if not os.path.exists(directory):
    os.makedirs(directory)


zz=i
counter=((zz)*h_num)+1
longitude_list=[start_long,start_long+long_dist]
counter

In [68]:
strt=datetime.datetime.today()
for i in range(zz,v_num):
    for j in range(h_num):
        lat=(latitude_list[0]+latitude_list[1])/2
        long=(longitude_list[0]+longitude_list[1])/2
        img_name='field_'+str(counter)+'_'+str(lat)+','+str(long)+'.png'
        
        img1 = open('temp_image1','wb')
        img2 = open('temp_image2','wb')
        img1.write(requests.get('https://maps.googleapis.com/maps/api/staticmap?center='+str(lat)+','+str(long)+'&zoom='+str(zoom)+'&size='+size+'&maptype=satellite&key='+str(api_key)).content)
        img2.write(requests.get('https://maps.googleapis.com/maps/api/staticmap?center='+str(lat-crop_section)+','+str(long)+'&zoom='+str(zoom)+'&size='+size+'&maptype=satellite&key='+str(api_key)).content)
        img1.close()
        img2.close()
        img1=cv2.imread('temp_image1')
        img2=cv2.imread('temp_image2')
        img1[-25:,:,:]=img2[-50:-25,:,:]
        
        cv2.imwrite('images/'+img_name,img1)
        
        """image = io.imread('images/'+img_name)
        plt.imshow(image)
        plt.show()"""
        
        print(counter)
        print(f"Center:{lat,long}\n Sides:\n{latitude_list[0],longitude_list[0]}\n {latitude_list[1],longitude_list[1]}")
        print('______________________________________')
        
        progressing(strt,counter-1,(h_num*v_num)-1)
        counter += 1
        
        longitude_list=[longitude_list[1] , longitude_list[1]+long_dist]
        #internal loop finished i.e. one row complete
        
    latitude_list=[latitude_list[1] , latitude_list[1]+lat_dist]
    longitude_list=[start_long,start_long+long_dist]

# ATTACHING IMAGES

In [74]:
folder_path='images/'
img_name_to_save='attached_image7.png'

In [ ]:

img_path=os.path.abspath(folder_path)
img_locs=[]
for i in os.listdir(img_path):
    img_locs.append(folder_path+i)

In [ ]:
img_locs.sort(key=lambda x: int(x.split('_')[1]))

# Attached Image (grayscale) for low ram

In [ ]:

final_image=np.zeros(( height*v_num , width*h_num))

imgs=[]
strt=datetime.datetime.today()
for sr,i in enumerate(img_locs):
    imgs.append(cv2.imread(i,0))
    progressing(strt,sr,len(img_locs))
    
counter=0
strt=datetime.datetime.today()
for j in range(v_num-1,-1,-1):
    for i in range(h_num):
        final_image[j*height:j*height+height,i*width:i*width+width]=imgs[counter]
        progressing(strt,counter,v_num*h_num)
        counter += 1


# Attached Image (colored) for high ram

In [ ]:

final_image=np.zeros(( height*v_num , width*h_num,3))

imgs=[]
strt=datetime.datetime.today()
for sr,i in enumerate(img_locs):
    imgs.append(cv2.imread(i))
    progressing(strt,sr,len(img_locs))
    
counter=0
strt=datetime.datetime.today()
for j in range(v_num-1,-1,-1):
    for i in range(h_num):
        final_image[j*height:j*height+height,i*width:i*width+width,:]=imgs[counter]
        progressing(strt,counter,v_num*h_num)
        counter += 1

In [82]:
#Saving Image
cv2.imwrite(img_name_to_save,final_image)

# IMAGE SEGMENTATION

In [ ]:
folder_to_load='images/'
folder_to_save='images-kmeans/'

directory = os.path.abspath(folder_to_save)
if not os.path.exists(directory):
    os.makedirs(directory)

img_path=os.path.abspath(folder)
img_locs=[]
for i in os.listdir(img_path):
    img_locs.append(os.path.join(i))

img_locs.sort(key=lambda x:int(x.split('_')[1]))


In [ ]:
strt=datetime.datetime.today()
for sr,i in enumerate(img_locs):
    img=cv2.imread(folder_to_load+i)
    x,y,z=img.shape
    Z=img.reshape(x*y,z)
    
    
    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 11
    ret,label,center=cv2.kmeans(Z,K,None,criteria,9,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    fin = res.reshape((img.shape))
    
    """km.fit(img)
    cluster_centers = km.cluster_centers_
    cluster_labels = km.labels_
    fin=cluster_centers[cluster_labels].reshape(x, y, z)"""
    
    cv2.imwrite(folder_to_save+i,fin)
    progressing(strt,sr,len(img_locs))
    
    #print(i.split('_')[1])

# Masking (getting threshold images)

In [ ]:
folder_to_load='images-kmeans/'
folder_to_save='images-kmeans-thresh/'

directory = os.path.abspath(folder_to_save)
if not os.path.exists(directory):
    os.makedirs(directory)


lower=np.array([20,70,10])
upper=np.array([110,220,230])

strt=datetime.datetime.today()
for sr,i in enumerate(img_locs):
    img=cv2.imread(folder_to_load+i)
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,lower,upper)
    
    cv2.imwrite(folder_to_save+i,mask)
    progressing(strt,sr,len(img_locs))

# Representing Top n Patch Area on Image

In [ ]:
#thresholded image
image_to_load='old_data/attached_image6.png'
image_to_save='patched4.png'
n=5 ##n= -0 for all

In [ ]:
img=cv2.imread(image_to_load,0)

"""##inverting image
mask0=img[img==0]
mask255=img[img==255]

img[mask0]=255
img[mask255]=0"""

im2, contours, hierarchy = cv2.findContours(img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x))
cnts_temp=[i for i in cntsSorted if cv2.contourArea(i) >=23500.0]

samp_img=cv2.imread(image_to_load)




#cv2.namedWindow('image', cv2.WINDOW_AUTOSIZE)
strt=datetime.datetime.today()
for sr,i in enumerate(cnts_temp[-n:]):
    cv2.drawContours(samp_img, [i], 0, (0,0,255), 2)
    M=cv2.moments(i)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    txt=str(np.round(cv2.contourArea(i)*px_area,decimals=3))+' sq. km ' ##contourarea gives number of pixels
    cv2.putText(samp_img,txt,(cx,cy),cv2.FONT_HERSHEY_DUPLEX,1.4,(55,0,255),1)
    progressing(strt,sr,len(cnts_temp[-n:]))
    """cv2.imshow('image',samp_img)
    cv2.waitKey(2)
cv2.waitKey(5000)
cv2.destroyAllWindows()"""

cv2.imwrite(image_to_save,samp_img)

# Getting Centroid points from image

In [ ]:
#threshold image
image_to_load='attached_image6.png'


In [ ]:
img=cv2.imread(image_to_load,0)

data=np.ones((1,3)) ##cx,cy,area

im2, contours, hierarchy = cv2.findContours(img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x))
cnts_temp=[i for i in cntsSorted if cv2.contourArea(i) >=23500.0]

samp_img=img.copy()


strt=datetime.datetime.today()
#cv2.namedWindow('image', cv2.WINDOW_AUTOSIZE)
for sr,i in enumerate(cnts_temp):
    #cv2.drawContours(samp_img, [i], 0, (0,0,255), 2)
    M=cv2.moments(i)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    ar=np.round(cv2.contourArea(i)*px_area,decimals=3)
    
    data=np.concatenate((data,np.array([cx,cy,ar]).reshape(1,-1)))
    #txt=str(np.round(cv2.contourArea(i)*px_area,decimals=3))+' sq. km ' ##contourarea gives number of pixels
    #cv2.putText(samp_img,txt,(cx,cy),cv2.FONT_HERSHEY_DUPLEX,1.4,(55,0,255),1)
    progressing(strt,sr,len(cnts_temp))

#cv2.imwrite('patched_grouping.png',samp_img)
data=data[1:]

In [ ]:
## Getting number of clusters from area, max area of each cluster should be 1500 acres i.e. 6.0702846 sq.km
max_ar=6.0702846
number_of_clusters=int(np.sum(data[:,2])//max_ar)
number_of_clusters

In [ ]:
img_height=img.shape[1]
img_height

In [ ]:
km=KMeans(n_clusters=number_of_clusters)
km.fit(data[:,:2])

# Plotting all Clusters

In [ ]:
plt.figure(figsize=(15,15))
color_list=['red','blue','yellow','pink','green','brown','cyan','gray']

for sr,i in enumerate(set(km.labels_)):
    cluster_label=np.where(km.labels_==i)[0]
    color=['black' if i==-1 else color_list[sr%len(color_list)]]
    plt.scatter(data[cluster_label,0],img_height-data[cluster_label,1],c=color,s=[data[cluster_label,2]/max(data[:,2])*10000])
    print(i, np.sum(data[cluster_label,2])*247 ,color)


# Plotting Clusters above 1500 acres area

In [ ]:
farms=[]

cluster_dict={}

min_area=1500.0

plt.figure(figsize=(15,15))
color_list=['red','blue','yellow','pink','green','brown','magenta','cyan','gray','lime']
counter=0
for sr,i in enumerate(set(km.labels_)):
    cluster_label=np.where(km.labels_==i)[0]
    color=['black' if i==-1 else color_list[sr%len(color_list)]]
    area=np.sum(data[cluster_label,2])*247
    if area >=min_area:
        counter += 1
        plt.scatter(data[cluster_label,0],img_height-data[cluster_label,1],c=color,s=[data[cluster_label,2]/max(data[:,2])*10000])
        plt.annotate(i,(km.cluster_centers_[i][0],img_height-km.cluster_centers_[i][1]))
        plt.annotate(i,(km.cluster_centers_[i][0],img_height-km.cluster_centers_[i][1]))
        farms.extend(list(cluster_label))
        cluster_dict[i]=cluster_label
        print(i,len(data[cluster_label,2]),area  ,color)
print('Total Clusters: ',counter)

##plotting DIC on map

dic_lat=int((end_point[0] - dic_coor[0])*111/px_lat)
dic_long=int((dic_coor[1] - start_point[1])*97/px_long)
plt.scatter(dic_lat,img_height-dic_long,c='black',s=1000,marker='*')

# Creating Dataframes

In [ ]:
df=pd.DataFrame(columns=['Origin_Address','Destination_Address','Origin_Coordinates','Destination_Coordinates','Duration','Distance','Cluster_Area(Acres)'])

df2=pd.DataFrame(columns=['Cluster_ID','Cluster_Name','Farm_Coordinates','Farm_Area'])

# Adding Data to Dataframes

In [ ]:
desti=str(dic_coor[0])+','+str(dic_coor[1])

strt=datetime.datetime.today()
for sr,i in enumerate(cluster_dict.keys()):
    lats=end_point[0] - (km.cluster_centers_[i,1]*px_lat)/111
    longs=(km.cluster_centers_[i,0]*px_long)/97 + start_point[1]
    area=np.sum(data[list(cluster_dict[i]),2])*247
    
    
    origin=str(lats)+','+str(longs)
    res=requests.get(
        'https://maps.googleapis.com/maps/api/distancematrix/json?units='+'metric'+
        '&origins='+origin+
        '&destinations='+desti+
        '&key='+api_key
    ).json()
    dur=res['rows'][0]['elements'][0]['duration']['text']
    dis=res['rows'][0]['elements'][0]['distance']['text']
    origin_addr=res['origin_addresses'][0]
    desti_addr=res['destination_addresses'][0]
    df.loc[i]=[origin_addr,desti_addr,origin,desti,dur,dis,area]
    progressing(strt,sr,len(cluster_dict.keys()))
    

In [ ]:
counter=0
total=0
for i in cluster_dict.keys():
    total += len(list(cluster_dict[i]))

strt=datetime.datetime.today()
for sr,i in enumerate(cluster_dict.keys()):
    for j in list(cluster_dict[i]):
        lats=end_point[0] - (data[j,1]*px_lat)/111
        longs=(data[j,0]*px_long)/97 + start_point[1]
        
        farm_coor=str(lats)+','+str(longs)
        farm_area=data[j,2]*247
        
        df2.loc[counter]=[i,df.loc[i,'Origin_Address'],farm_coor,farm_area]
        progressing(strt,counter,total)
        counter += 1
        

In [ ]:
distance=[]
for i in df.Distance.values:
    distance.append(float(i.split(' ')[0]))

df['Distance_derived']=distance
df['Distance_x_Area']=df['Cluster_Area(Acres)'] / distance

# Presenting Results Top n Recommendations

In [ ]:
n=6

In [ ]:
dd=df.nsmallest(n,columns='Distance_derived') #BAsed on min distance

In [ ]:
dd=df.nlargest(n,columns='Distance_x_Area') #Based on max of Area_per_distance

# Plotting Results

In [ ]:
plt.figure(figsize=(15,15))
plt.xlim(0,30000)
plt.ylim(0,30000)
for sr,i in enumerate(cluster_dict.keys()):
    cluster_label=list(cluster_dict[i])
    color=['black' if i==-1 else color_list[sr%len(color_list)]]
    if i in dd.index:
        mark='x'
    else:
        mark='o'
    plt.scatter(data[cluster_label,0],img_height-data[cluster_label,1],marker=mark,c=color,s=[data[cluster_label,2]/max(data[:,2])*10000])
    plt.annotate(i,(km.cluster_centers_[i][0],img_height-km.cluster_centers_[i][1]))
    print(i, np.sum(data[cluster_label,2])*247 ,color)

##plotting DIC on map

dic_lat=int((end_point[0] - dic_coor[0])*111/px_lat)
dic_long=int((dic_coor[1] - start_point[1])*97/px_long)
plt.scatter(dic_lat,img_height-dic_long,c='black',s=1000,marker='*')

#LAT -> 0.6099450000003728 km ==> (30.659516000000004 - 30.654021 )*111

#LONG -> 1.0432350000016868 km ==> (76.27289400000002 - 76.262139)*97

1px lat val (vertical) 0.0020331500000012426 km

#1px long val (horizontal) 0.0020864700000033736 km

#Area of 1px -> 4.2421064805094516e-06 km^2